In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as ans
import pandasql as ps
from sqlite3 import connect


#Reading Compustat
compustat = pd.read_csv('/Users/aaron/AaronTuFIMA/ErtimurReplication/Compustat2018Through2022.csv', parse_dates=True)
compustat.head(10)
Compustat2018Through2022 = compustat
print(Compustat2018Through2022.columns)


#Read crsp
crsp = pd.read_csv('/Users/aaron/AaronTuFIMA/ErtimurReplication/CRSP2020Through2022Daily.csv', parse_dates=True)
crsp.head(10)
Crsp2020Through2022 = crsp
print(Crsp2020Through2022.columns)

#Cleaning Compustat data
Compustat2018Through2022["datadate"] = pd.to_datetime(Compustat2018Through2022["datadate"], format = "%Y%m%d")
Compustat2018Through2022["rdq"] = pd.to_datetime(Compustat2018Through2022["rdq"], format = "%Y%m%d")
Compustat2018Through2022["cshoq"] = pd.to_numeric(Compustat2018Through2022["cshoq"], errors = 'coerce')
Compustat2018Through2022["ceqq"] = pd.to_numeric(Compustat2018Through2022["ceqq"], errors = 'coerce')
Compustat2018Through2022["ibq"] = pd.to_numeric(Compustat2018Through2022["ibq"], errors = 'coerce')
Compustat2018Through2022["saleq"] = pd.to_numeric(Compustat2018Through2022["saleq"], errors = 'coerce')
Compustat2018Through2022["prccq"] = pd.to_numeric(Compustat2018Through2022["prccq"], errors = 'coerce')

#indfmt = is character       #industry format
#datafmt = is character      #data format
Compustat2018Through2022['indfmt'] = Compustat2018Through2022['indfmt'].apply(str)
Compustat2018Through2022['datafmt'] = Compustat2018Through2022['datafmt'].apply(str)

#Correct formats
Compustat2018Through2022 = Compustat2018Through2022[Compustat2018Through2022["indfmt"]=="INDL"]
Compustat2018Through2022 = Compustat2018Through2022[Compustat2018Through2022["datafmt"]=="STD"]

#Only take dates after 2020
conn = connect(':memory:')

Compustat2018Through2022.to_sql("Compustat2018Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT *
    FROM Compustat2018Through2022
    WHERE Compustat2018Through2022.datadate>="20200101"
    '''
Compustat2020Through2022 = pd.read_sql(query, conn)
#print(tabulate(Compustat2020Through2022, headers = 'keys', tablefmt = 'presto'))

#renaming
compustat2020Through2022 = Compustat2020Through2022
crsp2020Through2022 = Crsp2020Through2022

#Clean CRSP data
crsp2020Through2022["date"] = pd.to_datetime(crsp2020Through2022["date"], format = "%Y%m%d")

#Get list of tickers with rdq dates from compustat
compustat2020Through2022.to_sql("compustat2020Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT compustat2020Through2022.tic, compustat2020Through2022.rdq
    FROM compustat2020Through2022
    WHERE compustat2020Through2022.tic<>"" and compustat2020Through2022.rdq<>""
    '''
compustat2020Through2022uniquegvkeysandrdqs = pd.read_sql(query, conn)
#print(tabulate(compustat2020Through2022uniquegvkeysandrdqs, headers = 'keys', tablefmt = 'presto'))

#Get tickers and dates from CRSP
crsp2020Through2022.to_sql("crsp2020Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT crsp2020Through2022.TICKER, crsp2020Through2022.date
    FROM crsp2020Through2022
    '''
crsp2020Through2022uniqueticsanddates = pd.read_sql(query, conn)
#print(tabulate(crsp2020Through2022uniqueticsanddates, headers = 'keys', tablefmt = 'presto'))



C:\Users\aaron\AppData\Local\Temp\ipykernel_24908\4213515657.py:24: DtypeWarning: Columns (17,18,26,30,647,669,674) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat = pd.read_csv('/Users/aaron/AaronTuFIMA/Compustat2018Through2022.csv', parse_dates=True)


Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'fyr', 'indfmt', 'consol',
       'popsrc', 'datafmt', 'tic',
       ...
       'priusa', 'sic', 'spcindcd', 'spcseccd', 'spcsrc', 'state', 'stko',
       'weburl', 'dldte', 'ipodate'],
      dtype='object', length=680)


C:\Users\aaron\AppData\Local\Temp\ipykernel_24908\4213515657.py:31: DtypeWarning: Columns (5,9,18,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp = pd.read_csv('/Users/aaron/AaronTuFIMA/CRSP2020Through2022Daily.csv', parse_dates=True)


Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP',
       'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT',
       'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT',
       'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG',
       'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM',
       'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD',
       'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD',
       'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')


In [2]:
#Left join tickers and dates from crsp with earnings dates from compustat
crsp2020Through2022uniqueticsanddates.to_sql("crsp2020Through2022uniqueticsanddates", conn, if_exists='replace')
compustat2020Through2022uniquegvkeysandrdqs.to_sql("compustat2020Through2022uniquegvkeysandrdqs", conn, if_exists='replace')
query = '''SELECT DISTINCT crsp2020Through2022uniqueticsanddates.TICKER, crsp2020Through2022uniqueticsanddates.date, compustat2020Through2022uniquegvkeysandrdqs.rdq
    FROM crsp2020Through2022uniqueticsanddates
    LEFT JOIN compustat2020Through2022uniquegvkeysandrdqs
    ON crsp2020Through2022uniqueticsanddates.TICKER = compustat2020Through2022uniquegvkeysandrdqs.tic
    and crsp2020Through2022uniqueticsanddates.date = compustat2020Through2022uniquegvkeysandrdqs.rdq
    ORDER BY crsp2020Through2022uniqueticsanddates.TICKER, crsp2020Through2022uniqueticsanddates.date
    '''
crspcompustat2020Through2022uniqueticsanddatesandrdq = pd.read_sql(query, conn)
print(crspcompustat2020Through2022uniqueticsanddatesandrdq.head(10))
#print(tabulate(crspcompustat2020Through2022uniqueticsanddatesandrdq, headers = 'keys', tablefmt = 'presto'))

  TICKER                 date   rdq
0   None  2020-01-02 00:00:00  None
1   None  2020-01-03 00:00:00  None
2   None  2020-01-06 00:00:00  None
3   None  2020-01-07 00:00:00  None
4   None  2020-01-08 00:00:00  None
5   None  2020-01-09 00:00:00  None
6   None  2020-01-10 00:00:00  None
7   None  2020-01-13 00:00:00  None
8   None  2020-01-14 00:00:00  None
9   None  2020-01-15 00:00:00  None


In [3]:
#KEEP ONLY THE ROWS THREE DAYS AFTER THE EARNINGS DATE
#sort the table dailyrat1004
crspcompustat2020Through2022uniqueticsanddatesandrdq.sort_values(by=['TICKER', 'date'], ascending = False)
print(crspcompustat2020Through2022uniqueticsanddatesandrdq.loc[0:1, :])

#calculating the 3 day return window
crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus1'] = crspcompustat2020Through2022uniqueticsanddatesandrdq.groupby(['TICKER'])['rdq'].shift(1)
crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus2'] = crspcompustat2020Through2022uniqueticsanddatesandrdq.groupby(['TICKER'])['rdq'].shift(2)
crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus3'] = crspcompustat2020Through2022uniqueticsanddatesandrdq.groupby(['TICKER'])['rdq'].shift(3)
print(crspcompustat2020Through2022uniqueticsanddatesandrdq.head(1))
crspcompustat2020Through2022uniqueticsanddatesandrdq['three_days_after_earnings_date_ifearningsduringday'] = crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus2']
crspcompustat2020Through2022uniqueticsanddatesandrdq['three_days_after_earnings_date_ifearningsafterhours'] = crspcompustat2020Through2022uniqueticsanddatesandrdq['rdq_plus3']
print(crspcompustat2020Through2022uniqueticsanddatesandrdq.head(1))

  TICKER                 date   rdq
0   None  2020-01-02 00:00:00  None
1   None  2020-01-03 00:00:00  None
  TICKER                 date   rdq rdq_plus1 rdq_plus2 rdq_plus3
0   None  2020-01-02 00:00:00  None       NaN       NaN       NaN
  TICKER                 date   rdq rdq_plus1 rdq_plus2 rdq_plus3  \
0   None  2020-01-02 00:00:00  None       NaN       NaN       NaN   

  three_days_after_earnings_date_ifearningsduringday  \
0                                                NaN   

  three_days_after_earnings_date_ifearningsafterhours  
0                                                NaN   


In [4]:
crspcompustat2020Through2022uniqueticsanddatesandrdq.to_sql("crspcompustat2020Through2022uniqueticsanddatesandrdq", conn, if_exists='replace')
query = '''SELECT DISTINCT crspcompustat2020Through2022uniqueticsanddatesandrdq.TICKER, crspcompustat2020Through2022uniqueticsanddatesandrdq.date,
    crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq, crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus1,
    crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus2, crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus3
    FROM crspcompustat2020Through2022uniqueticsanddatesandrdq
    WHERE crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq<>"NaN" or crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus1<>"NaN"
    or crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus2<>"NaN" or crspcompustat2020Through2022uniqueticsanddatesandrdq.rdq_plus3<>"NaN"
    '''
crspcompustat2020Through2022threedaysfollowingrdq = pd.read_sql(query, conn)
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdq, headers = 'keys', tablefmt = 'presto'))

In [5]:
print(crspcompustat2020Through2022threedaysfollowingrdq.head(10))
print(crsp2020Through2022.head(10))

  TICKER                 date                  rdq            rdq_plus1  \
0      A  2020-02-18 00:00:00  2020-02-18 00:00:00                 None   
1      A  2020-02-19 00:00:00                 None  2020-02-18 00:00:00   
2      A  2020-02-20 00:00:00                 None                 None   
3      A  2020-02-21 00:00:00                 None                 None   
4      A  2020-05-21 00:00:00  2020-05-21 00:00:00                 None   
5      A  2020-05-22 00:00:00                 None  2020-05-21 00:00:00   
6      A  2020-05-26 00:00:00                 None                 None   
7      A  2020-05-27 00:00:00                 None                 None   
8      A  2020-08-18 00:00:00  2020-08-18 00:00:00                 None   
9      A  2020-08-19 00:00:00                 None  2020-08-18 00:00:00   

             rdq_plus2            rdq_plus3  
0                 None                 None  
1                 None                 None  
2  2020-02-18 00:00:00              

In [ ]:
#Add the return data to the dates and ticker table
crspcompustat2020Through2022threedaysfollowingrdq.to_sql("crspcompustat2020Through2022threedaysfollowingrdq", conn, if_exists='replace')
crsp2020Through2022.to_sql("crsp2020Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT crspcompustat2020Through2022threedaysfollowingrdq.TICKER, crspcompustat2020Through2022threedaysfollowingrdq.date,
crspcompustat2020Through2022threedaysfollowingrdq.rdq, crsp2020Through2022.RET
    FROM crspcompustat2020Through2022threedaysfollowingrdq
    LEFT JOIN crsp2020Through2022
    ON crspcompustat2020Through2022threedaysfollowingrdq.TICKER = crsp2020Through2022.TICKER and
    crspcompustat2020Through2022threedaysfollowingrdq.date = crsp2020Through2022.date
    '''
crspcompustat2020Through2022threedaysfollowingrdqwithreturns = pd.read_sql(query, conn)
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdqwithreturns, headers = 'keys', tablefmt = 'presto'))

In [ ]:
#Add cash dividend payout YTD, sales, net income, shareholder equity, shares oustanding, price
crspcompustat2020Through2022threedaysfollowingrdqwithreturns.to_sql("crspcompustat2020Through2022threedaysfollowingrdqwithreturns", conn, if_exists='replace')
compustat2020Through2022.to_sql("compustat2020Through2022", conn, if_exists='replace')
query = '''SELECT DISTINCT crspcompustat2020Through2022threedaysfollowingrdqwithreturns.TICKER, crspcompustat2020Through2022threedaysfollowingrdqwithreturns.date,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturns.rdq, crspcompustat2020Through2022threedaysfollowingrdqwithreturns.RET,
    compustat2020Through2022.datadate, compustat2020Through2022.dvy, compustat2020Through2022.saleq, compustat2020Through2022.ibq,
    compustat2020Through2022.fyearq, compustat2020Through2022.fqtr, compustat2020Through2022.fyr, compustat2020Through2022.ceqq, compustat2020Through2022.cshoq,
    compustat2020Through2022.prccq
    FROM crspcompustat2020Through2022threedaysfollowingrdqwithreturns
    LEFT JOIN compustat2020Through2022
    ON crspcompustat2020Through2022threedaysfollowingrdqwithreturns.TICKER = compustat2020Through2022.tic and
    crspcompustat2020Through2022threedaysfollowingrdqwithreturns.rdq = compustat2020Through2022.rdq
    '''
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsanddatadate = pd.read_sql(query, conn)
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsanddatadate, headers = 'keys', tablefmt = 'presto'))
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr = crspcompustat2020Through2022threedaysfollowingrdqwithreturnsanddatadate


In [ ]:
#Add cumulative three day returns
table = crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr
table['ret_1'] = table.groupby(['TICKER'])['RET'].shift(-1)
table['ret_2'] = table.groupby(['TICKER'])['RET'].shift(-2)
table['ret_3'] = table.groupby(['TICKER'])['RET'].shift(-3)
table['RET'] = pd.to_numeric(table['RET'], errors = 'coerce')
table['ret_1'] = pd.to_numeric(table['ret_1'], errors = 'coerce')
table['ret_2'] = pd.to_numeric(table['ret_2'], errors = 'coerce')
table['ret_3'] = pd.to_numeric(table['ret_3'], errors = 'coerce')
table['three_day_return_ifearningsduringday'] = (1+table['RET']) + (1 + table['ret_1']) + (1 + table['ret_2']) - 1
table['three_day_return_ifearningsafterhours'] = (1+table['ret_1']) + (1 + table['ret_2']) + (1 + table['ret_3']) - 1
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr = table
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr, headers = 'keys', tablefmt = 'presto'))

In [ ]:
#Keep only the rows with an earnings announcement
crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.to_sql("crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr", conn, if_exists='replace')
query = '''SELECT DISTINCT crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.TICKER,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.date,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.rdq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.RET,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.datadate,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.dvy,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.saleq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ibq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.fyearq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.fqtr,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.fyr,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ceqq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.cshoq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.prccq,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ret_1,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ret_2,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.ret_3,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.three_day_return_ifearningsduringday,
    crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.three_day_return_ifearningsafterhours
    FROM crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr
    WHERE crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtr.rdq<>""
    '''

crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query, conn)
#print(tabulate(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq, headers = 'keys', tablefmt = 'presto'))
#print(crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.columns)

In [ ]:
#Create a table with 2019 data and the same rows
Compustat2018Through2022.to_sql("Compustat2018Through2022", conn, if_exists = 'replace')
query = '''SELECT DISTINCT tic as TICKER, rdq, datadate, dvy, saleq, ibq, fyearq, fqtr, fyr, ceqq, cshoq, prccq
    FROM Compustat2018Through2022
    WHERE Compustat2018Through2022.fyearq == "2019"
    ORDER BY TICKER, datadate
    '''

compustat2019ticrdqdatadatesaleqibqfyearqfqtrfyrceqqcshoqprccq = pd.read_sql(query, conn)

In [ ]:
#Append 2019 compustat to the main table
#Table2019to2022 = crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019 = compustat2019ticrdqdatadatesaleqibqfyearqfqtrfyrceqqcshoqprccq
Table2020to2022 = crspcompustat2020Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019to2022 = pd.concat([Table2019, Table2020to2022])

Table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending = False)
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = Table2019to2022


In [ ]:
#add market value of equity and book to market
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''SELECT DISTINCT TICKER, rdq, datadate, dvy, saleq, ibq, fyearq, fqtr, 
    fyr, ceqq, cshoq, prccq, date, RET, ret_1, ret_2, 
    ret_3, three_day_return_ifearningsduringday, three_day_return_ifearningsafterhours, cshoq*prccq as market_value_of_equity, 
    ceqq / (cshoq*prccq) as book_to_market_timet
    FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
    '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query, conn)


In [ ]:
#Rank the book to market for all companies at each fiscal quarter date and delineate the first third from the last third percentiles
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq["book_to_market_timet"] = crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq["book_to_market_timet"].apply(lambda x: 0.5 if x < 0 else x)
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists = 'replace')
query = '''SELECT DISTINCT
        TICKER, datadate, book_to_market_timet
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE book_to_market_timet<>""
        '''
booktomarketdata2019Through2022 = pd.read_sql(query, conn)
booktomarketdata2019Through2022['percentile'] = booktomarketdata2019Through2022.groupby(['datadate'])['book_to_market_timet'].rank(pct=True)
booktomarketdata2019Through2022.loc[booktomarketdata2019Through2022['percentile'] <= .33, 'FirstThird'] = True
booktomarketdata2019Through2022.loc[booktomarketdata2019Through2022['percentile'] >= .67, 'LastThird'] = True



In [ ]:
#rank the ytd dividend yield for all companies at each fiscal quarter date and delineate the first and last quartiles
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, dvy/market_value_of_equity as dividendyield
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE market_value_of_equity<>""
        ''' 
dividendyielddata2019Through2022 = pd.read_sql(query, conn)
dividendyielddata2019Through2022['percentile'] = dividendyielddata2019Through2022.groupby(['datadate'])['dividendyield'].rank(pct=True)
dividendyielddata2019Through2022.loc[dividendyielddata2019Through2022['percentile'] <= .25, 'FirstQuartile'] = True
dividendyielddata2019Through2022.loc[dividendyielddata2019Through2022['percentile'] >= .75, 'FourthQuartile'] = True

In [ ]:
#Merge book to market data back into the main table
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
booktomarketdata2019Through2022.to_sql("booktomarketdata2019Through2022", conn, if_exists='replace')
query = '''
        SELECT DISTINCT
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
        booktomarketdata2019Through2022.FirstThird as booktomarketfirstthird, booktomarketdata2019Through2022.LastThird as booktomarketlastthird
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        LEFT JOIN booktomarketdata2019Through2022
        ON crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER = booktomarketdata2019Through2022.TICKER
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate = booktomarketdata2019Through2022.datadate
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query, conn)
print(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.loc[1:10, :])

In [ ]:
#Merge the dividend yield data back into the main table
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
dividendyielddata2019Through2022.to_sql("dividendyielddata2019Through2022", conn, if_exists='replace')
query = '''
        SELECT DISTINCT
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketfirstthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketlastthird,
        dividendyielddata2019Through2022.FirstQuartile as dividendyieldfirstquartile, dividendyielddata2019Through2022.FourthQuartile as dividendyieldfourthquartile
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        LEFT JOIN dividendyielddata2019Through2022
        ON crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER = dividendyielddata2019Through2022.TICKER
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate = dividendyielddata2019Through2022.datadate
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query, conn)

In [ ]:
#add year ago quarterly sales and income
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketfirstthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketlastthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfirstquartile,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfourthquartile
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq<>""
        ORDER BY TICKER, datadate
        '''
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = pd.read_sql(query, conn)
print(crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.loc[1:10, :])

In [ ]:
#sort the Table
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.sort_values(by=['TICKER', 'datadate'], ascending=False)

Table2019to2022 = crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019to2022['saleq_4'] = Table2019to2022.groupby(['TICKER'])['saleq'].shift(4)
Table2019to2022['ibq_4'] = Table2019to2022.groupby(['TICKER'])['ibq'].shift(4)
Table2019to2022['fqtr_4'] = Table2019to2022.groupby(['TICKER'])['fqtr'].shift(4)
Table2019to2022['fyearq_4'] = Table2019to2022.groupby(['TICKER'])['fyearq'].shift(4)         
            
for row in Table2019to2022.index:
    try:
        #fq1 case
        if ((Table2019to2022['fqtr'][row] == 1.0) and (Table2019to2022['fyearq'][row] - 1 != Table2019to2022['fyearq_1'][row])) \
            or ((Table2019to2022['fqtr'][row] == 1.0) and (Table2019to2022['fqtr_1'][row] != 4.0)):
            Table2019to2022.at[row, 'fqtr_1'] = np.nan
            Table2019to2022.at[row, 'fyearq_1'] = np.nan
    except:
        continue
    try:
        #other cases
        if ((Table2019to2022['fqtr'][row] > 1.0) and (Table2019to2022['fyearq'][row] != Table2019to2022['fyearq_1'][row])) \
            or ((Table2019to2022['fqtr'][row] > 1.0) and (Table2019to2022['fqtr_1'][row] != Table2019to2022['fqtr'][row] - 1)):
            Table2019to2022.at[row, 'fqtr_1'] = np.nan
            Table2019to2022.at[row, 'fyearq_1'] = np.nan
    except:
        continue
        
Table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = Table2019to2022




In [ ]:
# Add quarter ago sales and income
Table2019to2022 = crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
Table2019to2022.sort_values(by=['TICKER', 'datadate'], ascending=False)

Table2019to2022['saleq_1'] = Table2019to2022.groupby(['TICKER'])['saleq'].shift(1)
Table2019to2022['ibq_1'] = Table2019to2022.groupby(['TICKER'])['ibq'].shift(1)
Table2019to2022['fqtr_1'] = Table2019to2022.groupby(['TICKER'])['fqtr'].shift(1)
Table2019to2022['fyearq_1'] = Table2019to2022.groupby(['TICKER'])['fyearq'].shift(1)

for row in Table2019to2022.index:
    try:
        if ((float(Table2019to2022['fqtr'][row])==1) and (float(Table2019to2022['fyearq'][row])==1)-1 != (float(Table2019to2022['fyearq_1'][row]))):
            Table2019to2022.loc[row, 'saleq_1'] = ""
    except:
        continue
    try:
        if ((float(Table2019to2022['fqtr'][row])==1) and (float(Table2019to2022['fyearq'][row])==1)-1 != (float(Table2019to2022['fyearq_1'][row]))):
            Table2019to2022.loc[row, 'ibq_1'] = ""
    except:
        continue
    try:
        if ((float(Table2019to2022['fqtr'][row])>1) and (float(Table2019to2022['fyearq'][row])) != (float(Table2019to2022['fyearq_1'][row]))):
            Table2019to2022.loc[row, 'fqtr_1'] = ""
    except:
        continue
    try:
        if ((float(Table2019to2022['fqtr'][row])>1) and (float(Table2019to2022['fyearq'][row])) != (float(Table2019to2022['fyearq_1'][row]))):
            Table2019to2022.loc[row, 'fyearq_1'] = ""
    except:
        continue

crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq = Table2019to2022


In [ ]:
#drop all rows with blank year ago and quarter ago sales and income
crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.to_sql("crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq", conn, if_exists='replace')
query = '''
        SELECT DISTINCT
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.TICKER,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.datadate,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dvy,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_4,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_4,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyr,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ceqq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.cshoq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.prccq,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.date,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.RET,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_1,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_2,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ret_3,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsduringday,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.market_value_of_equity,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.book_to_market_timet,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketfirstthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.booktomarketlastthird,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfirstquartile,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.dividendyieldfourthquartile,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fyearq_4,
        crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.fqtr_4
        FROM crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq
        WHERE crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.rdq<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_4<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_4<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.saleq_1<>""
        and crspcompustat2019Through2022threedaysfollowingrdqwithreturnsdatadatesaleqniqfyearqfyrfqtronlyrowswithnonnullrdq.ibq_1<>""
        '''
crspcompustat2019Through2022withsaleq4andibq4 = pd.read_sql(query, conn)
print(crspcompustat2019Through2022withsaleq4andibq4.loc[1:10, :])

In [ ]:
# Add in whether they are growth or value firms
Table2019to2022 = crspcompustat2019Through2022withsaleq4andibq4
Table2019to2022['growthfirm'] = ""
Table2019to2022['valuefirm'] = ""

for row in Table2019to2022.index:
    try:
        if (Table2019to2022['booktomarketfirstthird'][row] == 1) and (Table2019to2022['dividendyieldfourthquartile'][row] != 1):
            Table2019to2022.at[row, 'growthfirm'] = True
    except:
        continue
    try:
        if (Table2019to2022['booktomarketlastthird'][row] == 1) and (Table2019to2022['dividendyieldfourthquartile'][row] == 1):
            Table2019to2022.at[row, 'valuefirm'] = True
    except:
        continue

crspcompustat2019Through2022withsaleq4andibq4 = Table2019to2022


In [ ]:
#add seasonal random walk sales surprise and earnings surprise
Table2019to2022 = crspcompustat2019Through2022withsaleq4andibq4
Table2019to2022['saleq'] = pd.to_numeric(Table2019to2022['saleq'], errors = 'coerce')
Table2019to2022['saleq_4'] = pd.to_numeric(Table2019to2022['saleq_4'], errors = 'coerce')
Table2019to2022['ibq'] = pd.to_numeric(Table2019to2022['ibq'], errors = 'coerce')
Table2019to2022['ibq_4'] = pd.to_numeric(Table2019to2022['ibq_4'], errors = 'coerce')
Table2019to2022['srwsalessurprise'] = ((Table2019to2022['saleq']) - (Table2019to2022['saleq_4']))/(Table2019to2022['saleq_4'])
Table2019to2022['srwearningssurprise'] = ((Table2019to2022['ibq']) - (Table2019to2022['ibq_4']))/(Table2019to2022['ibq_4'])

crspcompustat2019Through2022withsaleq4andibq4 = Table2019to2022

In [ ]:
#add random walk sales surprise and earnings surprise
Table2019to2022 = crspcompustat2019Through2022withsaleq4andibq4
Table2019to2022['saleq'] = pd.to_numeric(Table2019to2022['saleq'], errors = 'coerce')
Table2019to2022['saleq_1'] = pd.to_numeric(Table2019to2022['saleq_1'], errors = 'coerce')
Table2019to2022['ibq'] = pd.to_numeric(Table2019to2022['ibq'], errors = 'coerce')
Table2019to2022['ibq_1'] = pd.to_numeric(Table2019to2022['ibq_1'], errors = 'coerce')
Table2019to2022['rwsalessurprise'] = ((Table2019to2022['saleq']) - (Table2019to2022['saleq_1']))/(Table2019to2022['saleq_1'])
Table2019to2022['rwearningssurprise'] = ((Table2019to2022['ibq']) - (Table2019to2022['ibq_1']))/(Table2019to2022['ibq_1'])

crspcompustat2019Through2022withsaleq4andibq4 = Table2019to2022
print(crspcompustat2019Through2022withsaleq4andibq4.head(10))

In [ ]:
ibessurprisehistory = pd.read_csv('/Users/aaron/AaronTuFIMA/ErtimurReplication/IBESSurpriseHistory.csv', parse_dates=True, index_col=0)
print(ibessurprisehistory.head(10))

In [ ]:
#calculate quarterly analyst sales and earnings surprise
import datetime
ibessurprisehistory = pd.read_csv('IBESSurpriseHistory.csv', parse_dates=True)
ibessurprisehistory['datadate'] = datetime.datetime(2010, 1, 1)
ibessurprisehistory['datadate'] = pd.to_datetime(ibessurprisehistory['datadate'], format="%Y%m%d")
ibessurprisehistory['PMON'] = pd.to_numeric(ibessurprisehistory['PMON'])
ibessurprisehistory['PYEAR'] = pd.to_numeric(ibessurprisehistory['PYEAR'])
ibessurprisehistory = ibessurprisehistory[ibessurprisehistory['PYEAR'] >= 2020]

for row in ibessurprisehistory.index:
    try:
        if ibessurprisehistory['PMON'][row] == 1:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] == 2 and ibessurprisehistory['PYEAR'][row] == 2020:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 29)
        elif ibessurprisehistory['PMON'][row] == 2 and ibessurprisehistory['PYEAR'][row] > 2020:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 28)
        elif ibessurprisehistory['PMON'][row] == 3:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] == 4:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 30)
        elif ibessurprisehistory['PMON'][row] == 5:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] == 6:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 30)
        elif ibessurprisehistory['PMON'][row] == 7:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] == 8:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] == 9:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 30)
        elif ibessurprisehistory['PMON'][row] == 10:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
        elif ibessurprisehistory['PMON'][row] == 11:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 30)
        elif ibessurprisehistory['PMON'][row] == 12:
            ibessurprisehistory.at[row, 'datadate'] = datetime.datetime(ibessurprisehistory['PYEAR'][row], ibessurprisehistory['PMON'][row], 31)
    except:
        continue

In [ ]:
#merge in analyst quarterly sales and income surprises
crspcompustat2019Through2022withsaleq4andibq4.to_sql("crspcompustat2019Through2022withsaleq4andibq4", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, rdq, three_day_return_ifearningsafterhours,
        srwsalessurprise, srwearningssurprise,
        rwsalessurprise, rwearningssurprise,
        growthfirm, valuefirm,
        book_to_market_timet, market_value_of_equity
        FROM crspcompustat2019Through2022withsaleq4andibq4
        WHERE three_day_return_ifearningsafterhours<>""
        '''
crspcompustat2019Through2022allfirms = pd.read_sql(query, conn)

In [ ]:
#add in the qtrepssuescore
ibessurprisehistory.to_sql("ibessurprisehistory", conn, if_exists='replace')
crspcompustat2019Through2022allfirms.to_sql("crspcompustat2019Through2022allfirms", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirms.TICKER, crspcompustat2019Through2022allfirms.datadate,
        crspcompustat2019Through2022allfirms.rdq, crspcompustat2019Through2022allfirms.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirms.srwsalessurprise, crspcompustat2019Through2022allfirms.srwearningssurprise,
        crspcompustat2019Through2022allfirms.rwsalessurprise, crspcompustat2019Through2022allfirms.rwearningssurprise,
        crspcompustat2019Through2022allfirms.growthfirm, crspcompustat2019Through2022allfirms.valuefirm,
        crspcompustat2019Through2022allfirms.book_to_market_timet, crspcompustat2019Through2022allfirms.market_value_of_equity,
        ibessurprisehistory.suescore as qtrepssuescore, ibessurprisehistory.anndats as qtrepsanndats
        FROM crspcompustat2019Through2022allfirms
        LEFT JOIN ibessurprisehistory
        ON crspcompustat2019Through2022allfirms.TICKER = ibessurprisehistory.OFTIC and 
        crspcompustat2019Through2022allfirms.datadate = ibessurprisehistory.datadate and 
        ibessurprisehistory.FISCALP == "QTR" and ibessurprisehistory.MEASURE == "EPS"
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistory = pd.read_sql(query, conn)


In [ ]:
#add in the qtrepssuescore
ibessurprisehistory.to_sql("ibessurprisehistory", conn, if_exists='replace')
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER, crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq, crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm, crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet, crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore, crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepsanndats,
        ibessurprisehistory.anndats as qtrsalesanndats, ibessurprisehistory.suescore as qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        LEFT JOIN ibessurprisehistory
        ON crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER = ibessurprisehistory.OFTIC and 
        crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate = ibessurprisehistory.datadate and 
        ibessurprisehistory.FISCALP == "QTR" and ibessurprisehistory.MEASURE == "SAL"
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistory = pd.read_sql(query, conn)

In [ ]:
#clean up the data and drop rows that don't have qtrepssuescore or qtrsalessuescore
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER, crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq, crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm, crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet, crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore, crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore<>"" and crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore<>""
        '''

crspcompustat2019Through2022allfirmswithibessurprisehistory = pd.read_sql(query, conn)

In [ ]:
#Drop all inf and na values 
crspcompustat2019Through2022allfirmswithibessurprisehistory.replace([np.inf, -np.inf], np.nan, inplace=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory = crspcompustat2019Through2022allfirmswithibessurprisehistory.dropna()

In [ ]:
#Calculate summary statistics for all firms

#calculate the number of unique firms
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE datadate >= "20200101"

        '''
numberofuniquefirms = pd.read_sql(query, conn)
print("Number of unique firms: " + str(len(numberofuniquefirms)))
print("\n")

#Earnings surprise SRW
print("SRW Earnings Surprise Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwearningssurprise'].min()))
print("SRW Earnings Surprise 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.1)))
print("SRW Earnings Surprise 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.25)))
print("SRW Earnings Surprise 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.50)))
print("SRW Earnings Surprise 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.75)))
print("SRW Earnings Surprise 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise.quantile(0.90)))
print("SRW Earnings Surprise Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwearningssurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwearningssurprise'])))
print("\n")

#Sales surprise SRW
print("SRW Sales Surprise Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwsalessurprise'].min()))
print("SRW Sales Surprise 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.1)))
print("SRW Sales Surprise 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.25)))
print("SRW Sales Surprise 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.50)))
print("SRW Sales Surprise 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.75)))
print("SRW Sales Surprise 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise.quantile(0.90)))
print("SRW Sales Surprise Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwsalessurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['srwsalessurprise'])))
print("\n")

#Earnings surprise RW
print("RW Earnings Surprise Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwearningssurprise'].min()))
print("RW Earnings Surprise 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.1)))
print("RW Earnings Surprise 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.25)))
print("RW Earnings Surprise 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.50)))
print("RW Earnings Surprise 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.75)))
print("RW Earnings Surprise 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise.quantile(0.90)))
print("RW Earnings Surprise Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwearningssurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwearningssurprise'])))
print("\n")

#Sales surprise RW
print("RW Sales Surprise Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwsalessurprise'].min()))
print("RW Sales Surprise 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.1)))
print("RW Sales Surprise 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.25)))
print("RW Sales Surprise 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.50)))
print("RW Sales Surprise 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.75)))
print("RW Sales Surprise 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise.quantile(0.90)))
print("RW Sales Surprise Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwsalessurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['rwsalessurprise'])))
print("\n")

#Book to market ratio
print("Book to Market Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['book_to_market_timet'].min()))
print("Book to Market 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.1)))
print("Book to Market 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.25)))
print("Book to Market 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.50)))
print("Book to Market 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.75)))
print("Book to Market 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet.quantile(0.90)))
print("Book to Market Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['book_to_market_timet'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['book_to_market_timet'])))
print("\n")

#Market Value of Equity
print("Market Value of Equity Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['market_value_of_equity'].min()))
print("Market Value of Equity 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.1)))
print("Market Value of Equity 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.25)))
print("Market Value of Equity 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.50)))
print("Market Value of Equity 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.75)))
print("Market Value of Equity 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity.quantile(0.90)))
print("Market Value of Equity Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['market_value_of_equity'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['market_value_of_equity'])))
print("\n")

#Analyst quarterly earnings surprise
print("Analyst quarterly earnings surprise Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescore'].min()))
print("Analyst quarterly earnings surprise 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.1)))
print("Analyst quarterly earnings surprise 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.25)))
print("Analyst quarterly earnings surprise 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.50)))
print("Analyst quarterly earnings surprise 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.75)))
print("Analyst quarterly earnings surprise 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore.quantile(0.90)))
print("Analyst quarterly earnings surprise Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescore'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescore'])))
print("\n")

#Analyst quarterly sales surprise
print("Analyst quarterly sales surprise Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescore'].min()))
print("Analyst quarterly sales surprise 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.1)))
print("Analyst quarterly sales surprise 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.25)))
print("Analyst quarterly sales surprise 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.50)))
print("Analyst quarterly sales surprise 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.75)))
print("Analyst quarterly sales surprise 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.90)))
print("Analyst quarterly sales surprise Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescore'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescore'])))
print("\n")

#Three Day Return
print("Three Day Return Min: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].min()))
print("Three Day Return 10th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.1)))
print("Three Day Return 25th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.25)))
print("Three Day Return 50th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.50)))
print("Three Day Return 75th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.75)))
print("Three Day Return 90th Percentile: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.90)))
print("Three Day Return Max: " + str(crspcompustat2019Through2022allfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022allfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'])))
print("\n")


In [ ]:
#summary statistics for all value firms

#select only value firms
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER, crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq, crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm, crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet, crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore, crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE valuefirm == 1
        '''
crspcompustat2019Through2022valuefirmswithibessurprisehistory = pd.read_sql(query, conn)



In [ ]:
#calculate the number of unique value firms
crspcompustat2019Through2022valuefirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022valuefirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER
        FROM crspcompustat2019Through2022valuefirmswithibessurprisehistory
        WHERE datadate >= "20200101"

        '''
numberofuniquefirms = pd.read_sql(query, conn)
print("Number of unique value firms: " + str(len(numberofuniquefirms)))
print("\n")

#Earnings surprise SRW
print("SRW Earnings Surprise Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwearningssurprise'].min()))
print("SRW Earnings Surprise 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.1)))
print("SRW Earnings Surprise 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.25)))
print("SRW Earnings Surprise 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.50)))
print("SRW Earnings Surprise 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.75)))
print("SRW Earnings Surprise 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwearningssurprise.quantile(0.90)))
print("SRW Earnings Surprise Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwearningssurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwearningssurprise'])))
print("\n")

#Sales surprise SRW
print("SRW Sales Surprise Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwsalessurprise'].min()))
print("SRW Sales Surprise 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.1)))
print("SRW Sales Surprise 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.25)))
print("SRW Sales Surprise 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.50)))
print("SRW Sales Surprise 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.75)))
print("SRW Sales Surprise 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.srwsalessurprise.quantile(0.90)))
print("SRW Sales Surprise Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwsalessurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['srwsalessurprise'])))
print("\n")

#Earnings surprise RW
print("RW Earnings Surprise Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwearningssurprise'].min()))
print("RW Earnings Surprise 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.1)))
print("RW Earnings Surprise 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.25)))
print("RW Earnings Surprise 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.50)))
print("RW Earnings Surprise 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.75)))
print("RW Earnings Surprise 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwearningssurprise.quantile(0.90)))
print("RW Earnings Surprise Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwearningssurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwearningssurprise'])))
print("\n")

#Sales surprise RW
print("RW Sales Surprise Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwsalessurprise'].min()))
print("RW Sales Surprise 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.1)))
print("RW Sales Surprise 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.25)))
print("RW Sales Surprise 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.50)))
print("RW Sales Surprise 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.75)))
print("RW Sales Surprise 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.rwsalessurprise.quantile(0.90)))
print("RW Sales Surprise Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwsalessurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['rwsalessurprise'])))
print("\n")

#Book to market ratio
print("Book to Market Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['book_to_market_timet'].min()))
print("Book to Market 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.1)))
print("Book to Market 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.25)))
print("Book to Market 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.50)))
print("Book to Market 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.75)))
print("Book to Market 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.book_to_market_timet.quantile(0.90)))
print("Book to Market Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['book_to_market_timet'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['book_to_market_timet'])))
print("\n")

#Market Value of Equity
print("Market Value of Equity Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['market_value_of_equity'].min()))
print("Market Value of Equity 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.1)))
print("Market Value of Equity 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.25)))
print("Market Value of Equity 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.50)))
print("Market Value of Equity 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.75)))
print("Market Value of Equity 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.market_value_of_equity.quantile(0.90)))
print("Market Value of Equity Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['market_value_of_equity'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['market_value_of_equity'])))
print("\n")

#Analyst quarterly earnings surprise
print("Analyst quarterly earnings surprise Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrepssuescore'].min()))
print("Analyst quarterly earnings surprise 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.1)))
print("Analyst quarterly earnings surprise 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.25)))
print("Analyst quarterly earnings surprise 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.50)))
print("Analyst quarterly earnings surprise 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.75)))
print("Analyst quarterly earnings surprise 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrepssuescore.quantile(0.90)))
print("Analyst quarterly earnings surprise Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrepssuescore'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrepssuescore'])))
print("\n")

#Analyst quarterly sales surprise
print("Analyst quarterly sales surprise Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrsalessuescore'].min()))
print("Analyst quarterly sales surprise 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.1)))
print("Analyst quarterly sales surprise 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.25)))
print("Analyst quarterly sales surprise 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.50)))
print("Analyst quarterly sales surprise 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.75)))
print("Analyst quarterly sales surprise 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.qtrsalessuescore.quantile(0.90)))
print("Analyst quarterly sales surprise Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrsalessuescore'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['qtrsalessuescore'])))
print("\n")

#Three Day Return
print("Three Day Return Min: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].min()))
print("Three Day Return 10th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.1)))
print("Three Day Return 25th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.25)))
print("Three Day Return 50th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.50)))
print("Three Day Return 75th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.75)))
print("Three Day Return 90th Percentile: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.90)))
print("Three Day Return Max: " + str(crspcompustat2019Through2022valuefirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022valuefirmswithibessurprisehistory['three_day_return_ifearningsafterhours'])))
print("\n")

In [ ]:
#summary statistics for all growth firms

#select only growth firms
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT crspcompustat2019Through2022allfirmswithibessurprisehistory.TICKER, crspcompustat2019Through2022allfirmswithibessurprisehistory.datadate,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rdq, crspcompustat2019Through2022allfirmswithibessurprisehistory.three_day_return_ifearningsafterhours,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.srwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.srwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.rwsalessurprise, crspcompustat2019Through2022allfirmswithibessurprisehistory.rwearningssurprise,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.growthfirm, crspcompustat2019Through2022allfirmswithibessurprisehistory.valuefirm,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.book_to_market_timet, crspcompustat2019Through2022allfirmswithibessurprisehistory.market_value_of_equity,
        crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrepssuescore, crspcompustat2019Through2022allfirmswithibessurprisehistory.qtrsalessuescore
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE growthfirm == 1
        '''
crspcompustat2019Through2022growthfirmswithibessurprisehistory = pd.read_sql(query, conn)



In [ ]:
#calculate the number of unique growth firms
crspcompustat2019Through2022growthfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022growthfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER
        FROM crspcompustat2019Through2022growthfirmswithibessurprisehistory
        WHERE datadate >= "20200101"

        '''
numberofuniquefirms = pd.read_sql(query, conn)
print("Number of unique growth firms: " + str(len(numberofuniquefirms)))
print("\n")

#Earnings surprise SRW
print("SRW Earnings Surprise Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwearningssurprise'].min()))
print("SRW Earnings Surprise 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.1)))
print("SRW Earnings Surprise 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.25)))
print("SRW Earnings Surprise 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.50)))
print("SRW Earnings Surprise 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.75)))
print("SRW Earnings Surprise 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwearningssurprise.quantile(0.90)))
print("SRW Earnings Surprise Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwearningssurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwearningssurprise'])))
print("\n")

#Sales surprise SRW
print("SRW Sales Surprise Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwsalessurprise'].min()))
print("SRW Sales Surprise 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.1)))
print("SRW Sales Surprise 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.25)))
print("SRW Sales Surprise 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.50)))
print("SRW Sales Surprise 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.75)))
print("SRW Sales Surprise 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.srwsalessurprise.quantile(0.90)))
print("SRW Sales Surprise Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwsalessurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['srwsalessurprise'])))
print("\n")

#Earnings surprise RW
print("RW Earnings Surprise Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwearningssurprise'].min()))
print("RW Earnings Surprise 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.1)))
print("RW Earnings Surprise 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.25)))
print("RW Earnings Surprise 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.50)))
print("RW Earnings Surprise 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.75)))
print("RW Earnings Surprise 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwearningssurprise.quantile(0.90)))
print("RW Earnings Surprise Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwearningssurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwearningssurprise'])))
print("\n")

#Sales surprise RW
print("RW Sales Surprise Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwsalessurprise'].min()))
print("RW Sales Surprise 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.1)))
print("RW Sales Surprise 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.25)))
print("RW Sales Surprise 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.50)))
print("RW Sales Surprise 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.75)))
print("RW Sales Surprise 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.rwsalessurprise.quantile(0.90)))
print("RW Sales Surprise Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwsalessurprise'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['rwsalessurprise'])))
print("\n")

#Book to market ratio
print("Book to Market Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['book_to_market_timet'].min()))
print("Book to Market 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.1)))
print("Book to Market 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.25)))
print("Book to Market 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.50)))
print("Book to Market 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.75)))
print("Book to Market 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.book_to_market_timet.quantile(0.90)))
print("Book to Market Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['book_to_market_timet'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['book_to_market_timet'])))
print("\n")

#Market Value of Equity
print("Market Value of Equity Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['market_value_of_equity'].min()))
print("Market Value of Equity 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.1)))
print("Market Value of Equity 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.25)))
print("Market Value of Equity 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.50)))
print("Market Value of Equity 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.75)))
print("Market Value of Equity 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.market_value_of_equity.quantile(0.90)))
print("Market Value of Equity Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['market_value_of_equity'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['market_value_of_equity'])))
print("\n")

#Analyst quarterly earnings surprise
print("Analyst quarterly earnings surprise Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrepssuescore'].min()))
print("Analyst quarterly earnings surprise 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.1)))
print("Analyst quarterly earnings surprise 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.25)))
print("Analyst quarterly earnings surprise 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.50)))
print("Analyst quarterly earnings surprise 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.75)))
print("Analyst quarterly earnings surprise 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrepssuescore.quantile(0.90)))
print("Analyst quarterly earnings surprise Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrepssuescore'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrepssuescore'])))
print("\n")

#Analyst quarterly sales surprise
print("Analyst quarterly sales surprise Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrsalessuescore'].min()))
print("Analyst quarterly sales surprise 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.1)))
print("Analyst quarterly sales surprise 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.25)))
print("Analyst quarterly sales surprise 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.50)))
print("Analyst quarterly sales surprise 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.75)))
print("Analyst quarterly sales surprise 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.qtrsalessuescore.quantile(0.90)))
print("Analyst quarterly sales surprise Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrsalessuescore'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['qtrsalessuescore'])))
print("\n")

#Three Day Return
print("Three Day Return Min: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].min()))
print("Three Day Return 10th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.1)))
print("Three Day Return 25th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.25)))
print("Three Day Return 50th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.50)))
print("Three Day Return 75th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.75)))
print("Three Day Return 90th Percentile: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory.three_day_return_ifearningsafterhours.quantile(0.90)))
print("Three Day Return Max: " + str(crspcompustat2019Through2022growthfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'].max()))
print("Number of Observations: " + str(len(crspcompustat2019Through2022growthfirmswithibessurprisehistory['three_day_return_ifearningsafterhours'])))
print("\n")

In [ ]:
#winsorize the variables (drop 1% and 99% extremes)
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'] = crspcompustat2019Through2022allfirmswithibessurprisehistory.groupby(['datadate'])['qtrepssuescore'].rank(pct=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorefirstpercentile'] = ""
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescoreninetyninthpercentile'] = ""

for row in crspcompustat2019Through2022allfirmswithibessurprisehistory.index:
    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row] <= 0.01:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrepssuescorefirstpercentile'] = 1
    except:
        continue

    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row] >= 0.99:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrepssuescoreninetyninthpercentile'] = 1
    except:
        continue

    try:
        if 0.01 < crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrepssuescorepercentile'][row] < 0.99:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrepssuescorefirstpercentile'] = 0
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrepssuescoreninetyninthpercentile'] = 0
    except:
        continue


In [ ]:
#winsorize the qtrsalessuescore
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'] = crspcompustat2019Through2022allfirmswithibessurprisehistory.groupby(['datadate'])['qtrsalessuescore'].rank(pct=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorefirstpercentile'] = ""
crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescoreninetyninthpercentile'] = ""

for row in crspcompustat2019Through2022allfirmswithibessurprisehistory.index:
    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row] <= 0.01:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrsalessuescorefirstpercentile'] = 1
    except:
        continue

    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row] >= 0.99:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrsalessuescoreninetyninthpercentile'] = 1
    except:
        continue

    try:
        if 0.01 < crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row] < 0.99:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrsalessuescorefirstpercentile'] = 0
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'qtrsalessuescoreninetyninthpercentile'] = 0
    except:
        continue


In [ ]:
#winsorize the three day returns
crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnpercentile'] = crspcompustat2019Through2022allfirmswithibessurprisehistory.groupby(['datadate'])['three_day_return_ifearningsafterhours'].rank(pct=True)
crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnfirstpercentile'] = ""
crspcompustat2019Through2022allfirmswithibessurprisehistory['threedayreturnninetyninthpercentile'] = ""

for row in crspcompustat2019Through2022allfirmswithibessurprisehistory.index:
    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row] <= 0.01:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'threedayreturnfirstpercentile'] = 1
    except:
        continue

    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row] >= 0.99:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'threedayreturnninetyninthpercentile'] = 1
    except:
        continue

    try:
        if 0.01 < crspcompustat2019Through2022allfirmswithibessurprisehistory['qtrsalessuescorepercentile'][row] < 0.99:
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'threedayreturnfirstpercentile'] = 0
            crspcompustat2019Through2022allfirmswithibessurprisehistory.loc[row, 'threedayreturnninetyninthpercentile'] = 0
    except:
        continue

In [ ]:
#winsorize or remove the 1% and 99% extremes
crspcompustat2019Through2022allfirmswithibessurprisehistory.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistory", conn, if_exists='replace')
query = '''
        SELECT DISTINCT TICKER, datadate, rdq, three_day_return_ifearningsafterhours, srwsalessurprise,
        srwearningssurprise, rwsalessurprise, rwearningssurprise, 
        qtrepssuescore, qtrsalessuescore, growthfirm, valuefirm, market_value_of_equity,
        qtrepssuescorefirstpercentile, qtrepssuescoreninetyninthpercentile, qtrsalessuescorefirstpercentile,
        qtrsalessuescoreninetyninthpercentile, threedayreturnfirstpercentile, threedayreturnninetyninthpercentile
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistory
        WHERE three_day_return_ifearningsafterhours<>"" and qtrepssuescore<>"" and qtrsalessuescore<>"" and qtrepssuescorefirstpercentile<>1
        and qtrepssuescoreninetyninthpercentile<>1 and qtrsalessuescorefirstpercentile<>1 and qtrsalessuescoreninetyninthpercentile<>1
        and threedayreturnfirstpercentile<>1 and threedayreturnninetyninthpercentile<>1
        '''
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore = pd.read_sql(query, conn)


In [ ]:
#run a regression of the three day return variable on anaylist quarterly earnings and sales surprises
import statsmodels.formula.api as sm
from scipy.stats.mstats import winsorize
import pandas
import statsmodels.api as sm

#save to the file
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.to_csv("crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.csv")

#keep only the 2020 data
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.to_sql("crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore", conn, if_exists='replace')
query = '''
        SELECT DISTINCT *
        FROM crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore
        WHERE datadate >= "2020-01-01"
        '''

crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore = pd.read_sql(query, conn)

In [ ]:
#Reproduce table 2 panel A
threedayreturnmodeltable2 = ols("three_day_return_ifearningsafterhours ~ qtrsalessuescore", data = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()

#summarize our model
threedayreturnmodeltable2_summary = threedayreturnmodeltable2.summary()
print(threedayreturnmodeltable2_summary)

threedayreturnmodeltable22 = ols("three_day_return_ifearningsafterhours ~ qtrepssuescore", data = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()

#summarize our model 
threedayreturnmodeltable22_summary = threedayreturnmodeltable22.summary()
print(threedayreturnmodeltable22_summary)


In [ ]:
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrexpsuescore'] = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore'] - crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore'] 
print(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.head(10))

In [ ]:
#table 2 continued
threedayreturnmodeltable222 = ols("three_day_return_ifearningsafterhours ~ srwearningssurprise", data = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()

#summarize our model
threedayreturnmodeltable222_summary = threedayreturnmodeltable222.summary()
print(threedayreturnmodeltable222_summary)

In [ ]:
#run a regression of three day returns on value firm dummy, analyst quarterly earnings surprise, analyst quarterly
#sales surprise, analyst quarterly earnings surprise interacted with value dummy, 
# analyst quarterly sales surprise interacted with value dummy

test = False

crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuefirm'] = pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuefirm'], errors = 'coerce')
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'] = ""
for row in crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.index:
    try:
        if crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.loc[row, 'valuefirm'] == 1:
            crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.loc[row, 'valuedummy'] = 1
            test = True
        else:
            crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore.loc[row, 'valuedummy'] = 0
    except:
        continue

crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore'] = pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore'], errors = 'coerce') 
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'] = pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy'], errors = 'coerce') 
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore'] = pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore'], errors = 'coerce') 
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrexpsuescore'] = pd.to_numeric(crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrexpsuescore'], errors = 'coerce') 

crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescoreXvaluedummy'] = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrsalessuescore'] * crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescoreXvaluedummy'] = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrepssuescore'] * crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']
crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrexpsuescoreXvaluedummy'] = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['qtrexpsuescore'] * crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore['valuedummy']



In [ ]:
#Reproduce table 3
#threedayreturnmodeltable3 = ols("three_day_return_ifearningsafterhours ~ valuedummy + qtrsalessuescore + qtrepssuescore + qtrsalessuescoreXvaluedummy + qtrepssuescoreXvaluedummy", data = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()
threedayreturnmodeltable3 = ols("three_day_return_ifearningsafterhours ~ qtrsalessuescore + valuedummy + qtrsalessuescoreXvaluedummy", data = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()
threedayreturnmodeltable3_summary = threedayreturnmodeltable3.summary()
print(threedayreturnmodeltable3_summary)



In [ ]:
threedayreturnmodeltable3 = ols("three_day_return_ifearningsafterhours ~ qtrsalessuescore + valuedummy + qtrsalessuescoreXvaluedummy + qtrexpsuescore + qtrexpsuescoreXvaluedummy", data = crspcompustat2019Through2022allfirmswithibessurprisehistoryforqtrepssuescoreregressionandqtrsalessuescore).fit()
threedayreturnmodeltable3_summary = threedayreturnmodeltable3.summary()
print(threedayreturnmodeltable3_summary)